# Regridding simulated spread of Uranium 236

## Description
The data has been provided by NorESM, on a tripolar grid ("ds_hague" and "ds" datasets in this notebook).  
To arrive at a DGGS grid, the data has first been regridded from tripolar to PlateCarree grid ("dr" dataset in this notebook) using cdo:  

```console
brew install netcdf  
brew install nco  
```

Append (-A) the variables plat and plon from grid.nc into U236LH.nc:  

```console
ncks -A -v plat,plon ./data/grid/grid.nc ./data/model/JRAOC20TRNRPv2_hm_U236LH_2010-2018.nc  
```

Bilinear regridding:  
```console
cdo -O remapbil,global_1 ./data/model/JRAOC20TRNRPv2_hm_U236LH_2010-2018.nc U236LH_1x1d_bilinear.nc  
```

Different sources/scenarios of Uranium 236 spread are simulated: 
- Global fallout (GF)  
- Sellafield (SF)  
- La Hague (LH)  

### This notebook step by step:
1. Load required libraries
2. Load Uranium 236 data on tripolar grid and visualize
3. Load regridded dataset (PlateCarree grid)
4. Define DGGS target grid and regrid from PlateCarree to DGGS
5. Save the regridded data to zarr

## Contributions
- Even Moa Myklebust, Simula Research Laboratory (Norway) (author), @evenmm

## Bibliography and other interesting resources
- [The Norwegian Earth System Model (NorESM)](https://noresm-docs.readthedocs.io/en/latest/)
- [An introduction to NorESM model output and post-processing](https://nordicesmhub.github.io/noresmdiagnostics/)

In [ ]:
# Install xarray-healpy and dggs libraries for regridding
%pip install git+https://github.com/IAOCEA/xarray-healpy.git git+https://github.com/xarray-contrib/xdggs.git

In [ ]:
import warnings
from pathlib import Path

import cartopy.crs as ccrs  # Map projections
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr  # N-dimensional arrays with dimension, coordinate and attribute labels

# Local imports
from data_handling import load_grid_vertex, regrid_to_dggs, standardize_variable_names

warnings.simplefilter("ignore", category=DeprecationWarning)
xr.set_options(display_expand_data=False, display_expand_attrs=False, keep_attrs=True)

In [ ]:
# Get grid location information
data_path = Path("./CS1-nird/data/")
grid_path = data_path / "grid" / "grid.nc"
plat, plon, pclat, pclon = load_grid_vertex(grid_path)

## Uranium 236 from La Hague (ds_hague) on tripolar grid

In [ ]:
# Open and look at tripolar data from source grid
la_hague_dataset_path = data_path / "model" / "JRAOC20TRNRPv2_hm_U236LH_2010-2018.nc"
ds_hague = xr.open_dataset(la_hague_dataset_path)

In [ ]:
# Plot on tripolar grid naively using x and y
# Sigma refers to vertical level in the Sigma coordinate system
ds_hague.U236LH.isel(time=100, sigma=0).plot()

## Uranium 236 from Global fallout (ds) on tripolar grid

In [ ]:
global_fallout_dataset_path = (
    data_path / "model" / "JRAOC20TRNRPv2_hm_U236GF_2010-2018.nc"
)
ds = xr.open_dataset(global_fallout_dataset_path)
ds

In [ ]:
# Plot U236GF on tripolar grid naively, using x and y
ds.U236GF.isel(time=100, sigma=0).plot()

In [ ]:
# Use grid.nc to define latitude and longitude
ds = ds.assign_coords(lat=(["y", "x"], plat), lon=(["y", "x"], plon))

In [ ]:
# Standardize variable names
ds = standardize_variable_names(ds)
ds

In [ ]:
proj = ccrs.NearsidePerspective(
    central_longitude=0.0, central_latitude=80.0, satellite_height=3e6
)
fig, ax = plt.subplots(1, figsize=(8, 4.5), dpi=96, subplot_kw={"projection": proj})

# A temperature map
pm0 = ax.pcolormesh(
    plon,
    plat,
    ds.U236GF[0, 0, :, :],
    # vmin=0,
    # vmax=20,
    cmap="viridis",
    transform=ccrs.PlateCarree(),
    shading="auto",
    rasterized=True,
)

# Add coastlines and the lat-lon grid
ax.coastlines(resolution="50m", color="black", linewidth=0.5)
ax.stock_img()
gl = ax.gridlines(ylocs=range(15, 76, 15), draw_labels=True)
gl.ylocator = mpl.ticker.FixedLocator([40, 50, 60, 70, 80])

plt.colorbar(pm0, fraction=0.2, shrink=0.4, label="kg-1")

ax.set_title("Global Fallout Uranium 236")
plt.show()

# Regridded to PlateCaree: Uranium 236 from Global fallout (dr)

In [ ]:
# regridded_dataset_path = data_path / "U236GF_1x1d_bilinear.nc"
# regridded_dataset_path = data_path / "JRAOC20TRNRPv2_hm_U236GF_2010-2018_1x1d_bil.nc"
regridded_dataset_path = (
    data_path / "model" / "JRAOC20TRNRPv2_hm_U236GF_2010-2018_bil.nc"
)

dr = xr.open_dataset(regridded_dataset_path)
dr

In [ ]:
dr = dr.rename_dims({"lat": "latitude", "lon": "longitude"})
dr.latitude.attrs["standard_name"] = "latitude"
dr.longitude.attrs["standard_name"] = "longitude"
dr[["longitude", "latitude"]].compute()
dr = dr.rename({"lon": "longitude", "lat": "latitude"})

In [ ]:
# Bilinear regridding
dr.U236GF.isel(time=0, sigma=0).plot()

In [ ]:
# On the regridded data, we can plot parts of it like this:

# Define zoomed-in region (adjust as needed)
lat_min, lat_max = 40, 65  # Example range
lon_min, lon_max = -15, 30

# Select only the region of interest
dr_zoomed = dr.U236GF.isel(time=0, sigma=0).sel(
    latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max)
)

# Plot the zoomed-in region
dr_zoomed.plot()

In [ ]:
# Get grid location information
grid_path = data_path / "grid" / "grid.nc"
plat, plon, pclat, pclon = load_grid_vertex(grid_path)

# Conservative 

In [ ]:
conservative_regridded_dataset_path = (
    # data_path / "model" / "JRAOC20TRNRPv2_hm_U236GF_2010-2018_1x1d_con.nc"
    data_path
    / "model"
    / "JRAOC20TRNRPv2_hm_U236GF_2010-2018_con.nc"
)
dcon = xr.open_dataset(conservative_regridded_dataset_path)
dcon

In [ ]:
dcon = dcon.rename_dims({"lat": "latitude", "lon": "longitude"})
dcon.latitude.attrs["standard_name"] = "latitude"
dcon.longitude.attrs["standard_name"] = "longitude"
dcon[["longitude", "latitude"]].compute()
dcon = dcon.rename_vars({"lon": "longitude", "lat": "latitude"})

In [ ]:
# Conservative regridding
dcon.U236GF.isel(time=0, sigma=0).plot()

In [ ]:
regrid_diff = dr - dcon
regrid_diff

In [ ]:
print(
    np.max(regrid_diff), np.min(regrid_diff), np.max(regrid_diff), np.min(regrid_diff)
)

In [ ]:
regrid_diff.U236GF.isel(time=0, sigma=0).plot()

# Regrid to DGGS

In [ ]:
nside = (
    32  # 16 # Each side of the original 12 faces in Healpix is divided into nside parts
)
healpy_grid_level = int(np.log2(nside))  # Healpix level
number_of_cells = 12 * nside**2  # The resulting total number of cells

min_vertices = 1  # Minimum number of vertices for a valid transcription for regridding.
# 1 is the most liberal, meaning that only one is needed

print("nside:", nside)
print("Level:", healpy_grid_level)
print("Number of cells:", number_of_cells)

In [ ]:
# Perform the actual regridding
# regridded = regrid_to_dggs(dr, nside, min_vertices, method="bilinear", mask=None)
regridded = regrid_to_dggs(dcon, nside, min_vertices, method="bilinear", mask=None)

In [ ]:
# Compute to be able to visualize it
ds_regridded = regridded.U236GF.compute().squeeze()

In [ ]:
# Visualize
ds_regridded.dggs.explore()

# 5. Save the regridded data to zarr

In [ ]:
save_location = data_path / f"U236GF-healpix-lvl-{healpy_grid_level}.zarr"
ds_regridded.to_zarr(save_location, mode="w")